Async Connection

In [ ]:
EVENT_HUB_CONN_STR='Endpoint=sb://factored-datathon.servicebus.windows.net/;SharedAccessKeyName=datathon_group_1;SharedAccessKey=2GETvVt0FxyM0bo0Qau4inlmC/w3t4Uut+AEhAnAEgk=;EntityPath=factored_datathon_amazon_reviews_1'
EVENTHUB_NAME="factored_datathon_amazon_reviews_1"
CONSUMER_GROUP="datacticos"

In [ ]:
import logging
import asyncio
from azure.eventhub.aio import EventHubConsumerClient

import nest_asyncio
nest_asyncio.apply()

logger = logging.getLogger("azure.eventhub")
logging.basicConfig(level=logging.INFO)

output_file = "output_file_2.txt"

async def on_event(partition_context, event):
    logger.info("Received event from partition {}".format(partition_context.partition_id))
    print("Received event from partition {}".format(partition_context.partition_id))
    
    with open(output_file, "a+") as file:
        file.write(f"{list(event.body)}\n")

    await partition_context.update_checkpoint(event)

async def receive():
    client =  EventHubConsumerClient.from_connection_string(conn_str=EVENT_HUB_CONN_STR,consumer_group=CONSUMER_GROUP,eventhub_name=EVENTHUB_NAME,)
    async with client:
        await client.receive(
            on_event=on_event,
            starting_position="-1",  # "-1" is from the beginning of the partition.
        )
        # receive events from specified partition:
        # await client.receive(on_event=on_event, partition_id='0')

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.run_until_complete(receive())